In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio

import geopandas 
import json

In [2]:
geosonNeighbourhoods = geopandas.read_file("../../Data/neighbourhoods/neighbourhoods.geojson")
jsonGeoNeigh = json.load(open("../../Data/neighbourhoods/neighbourhoods.geojson"))

In [3]:
geosonNeighbourhoods.head()

,neighbourhood,neighbourhood_group,geometry
0,Bayswater,Queens,"MULTIPOLYGON (((-73.76671 40.61491, -73.76825 ..."
1,Allerton,Bronx,"MULTIPOLYGON (((-73.84860 40.87167, -73.84582 ..."
2,City Island,Bronx,"MULTIPOLYGON (((-73.78282 40.84392, -73.78257 ..."
3,Ditmars Steinway,Queens,"MULTIPOLYGON (((-73.90160 40.76777, -73.90391 ..."
4,Ozone Park,Queens,"MULTIPOLYGON (((-73.83754 40.69136, -73.83108 ..."


In [4]:
geosonNeighbourhoods

,neighbourhood,neighbourhood_group,geometry
0,Bayswater,Queens,"MULTIPOLYGON (((-73.76671 40.61491, -73.76825 ..."
1,Allerton,Bronx,"MULTIPOLYGON (((-73.84860 40.87167, -73.84582 ..."
2,City Island,Bronx,"MULTIPOLYGON (((-73.78282 40.84392, -73.78257 ..."
3,Ditmars Steinway,Queens,"MULTIPOLYGON (((-73.90160 40.76777, -73.90391 ..."
4,Ozone Park,Queens,"MULTIPOLYGON (((-73.83754 40.69136, -73.83108 ..."
...,...,...,...
228,Westchester Square,Bronx,"MULTIPOLYGON (((-73.84294 40.84640, -73.83913 ..."
229,Westerleigh,Staten Island,"MULTIPOLYGON (((-74.13131 40.62633, -74.13192 ..."
230,Williamsbridge,Bronx,"MULTIPOLYGON (((-73.86722 40.86969, -73.86701 ..."
231,Williamsburg,Brooklyn,"MULTIPOLYGON (((-73.95757 40.72510, -73.95300 ..."


In [5]:
# from https://medium.com/@rfeers/how-to-merge-polygons-using-geopandas-for-python-merging-polygong-data-science-catalonia-7fe15eef2b14
def boundary_generator(gdf,string):
    """Devuelve el multipolígono de todos los barrios pequeños

    Args:
        gdf (GeoDataFrame): Geo DataFrame con barrios pequeños, grandes y una geometria
        string (str): Nombre del barrio grande

    Returns:
        MULTYPOLYGON: multipolygono suma
    """
    polygons = gdf.loc[(gdf['neighbourhood_group'] == string)]
    polygons = polygons['geometry'].unary_union
    return polygons


In [6]:
listaNeighbours = []
for index, geofila in geosonNeighbourhoods.iterrows():
    listaNeighbours.append(geofila.neighbourhood_group)
barriosDistintos = list(set(listaNeighbours))
barriosDistintos

['Bronx', 'Staten Island', 'Brooklyn', 'Manhattan', 'Queens']

In [7]:
fronteras = []
for barrio in barriosDistintos:
    fronteras.append(boundary_generator(geosonNeighbourhoods,barrio))
fronteras

In [8]:
df = pd.DataFrame()
df["neighbourhood_group"] = barriosDistintos
df["geometry"] = fronteras
df

C:\Users\nacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\dtypes\cast.py:1990: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
C:\Users\nacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\dtypes\inference.py:383: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  iter(obj)  # Can iterate over it.
C:\Users\nacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\dtypes\inference.py:384: ShapelyDepreca

,neighbourhood_group,geometry
0,Bronx,"(POLYGON ((-73.890151 40.804958, -73.889374 40..."
1,Staten Island,"POLYGON ((-74.213382 40.506766, -74.213531 40...."
2,Brooklyn,"POLYGON ((-73.982016 40.571388, -73.981992 40...."
3,Manhattan,"(POLYGON ((-73.999551 40.707694, -73.999609 40..."
4,Queens,"(POLYGON ((-73.862694 40.566502, -73.862704 40..."


In [9]:
gdf = geopandas.GeoDataFrame(df,crs="EPSG:4326")

In [10]:
gdf

,neighbourhood_group,geometry
0,Bronx,"MULTIPOLYGON (((-73.89015 40.80496, -73.88937 ..."
1,Staten Island,"POLYGON ((-74.21338 40.50677, -74.21353 40.506..."
2,Brooklyn,"POLYGON ((-73.98202 40.57139, -73.98199 40.571..."
3,Manhattan,"MULTIPOLYGON (((-73.99955 40.70769, -73.99961 ..."
4,Queens,"MULTIPOLYGON (((-73.86269 40.56650, -73.86270 ..."


In [11]:
gdf["geometry"].centroid

C:\Users\nacho\AppData\Local\Temp/ipykernel_8032/3174659127.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["geometry"].centroid


0    POINT (-73.87173 40.85038)
1    POINT (-74.15259 40.58141)
2    POINT (-73.95232 40.64822)
3    POINT (-73.96796 40.77819)
4    POINT (-73.82121 40.71408)
dtype: geometry

In [27]:
gdf.to_file('../../Data/neighbourhoods/bigneighbourhoods.geojson', driver='GeoJSON')
